In [1]:
import numpy as np
import cv2 
from time import time
from os.path import join
import random, os
from tqdm import tqdm


CASC_PATH = join('./','haarcascade_files','haarcascade_frontalface_default.xml')
cascade_classifier = cv2.CascadeClassifier(CASC_PATH)
    

def detect_face(image, return_max_area_face=True):
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.imdecode(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
    faces = cascade_classifier.detectMultiScale(
        image,
        scaleFactor=1.3,
        minNeighbors=5
    )
    # None is we don't found an image
    if not len(faces) > 0:
        return []
    if return_max_area_face:
        max_area_face = faces[0]
        for face in faces:
            if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
                max_area_face = face
        faces = [max_area_face, ]

    return faces

def bounding_face(frame, faces, factor, constraint=None):
    bounding_box_list = []
    
    for face in faces:
        x,y,w,h = face
        factor = 2.
        offset = (factor-1)/2.
        rect = [x-offset*w, y-offset*h, w*factor, h*factor]
        for i,a in enumerate(rect):
            rect[i] = int(a)
        rect = tuple(rect)



        x,y = rect[:2]
        x2 = x + rect[2]
        y2 = y + rect[3]
        box = (int(x), int(y)), (int(x2), int(y2))
        box = shift_box(frame, box)
        if box:
        
            if constraint:
                if constraint(box):
                    bounding_box_list.append(box)
            else:
                bounding_box_list.append(box)
    return bounding_box_list
def bounding_face_fixed_size(frame, faces, fixed_size, constraint=None):
    bounding_box_list = []
    for face in faces:
        x,y,w,h = face
        offset_x = (fixed_size-w)/2.
        offset_y = (fixed_size-h)/2.
        rect = [x-offset_x, y-offset_y, fixed_size, fixed_size]
        for i,a in enumerate(rect):
            rect[i] = int(a)
        rect = tuple(rect)

        

        x,y = rect[:2]
        x2 = x + rect[2]
        y2 = y + rect[3]
        box = (int(x), int(y)), (int(x2), int(y2))
        box = shift_box(frame, box)
        if box:
            if constraint:
                if constraint(box):
                    bounding_box_list.append(box)
            else:
                bounding_box_list.append(box)
    return bounding_box_list
# handle the situation that box boundary reachs the boundary of frame. ex, y<0
def shift_box(frame, box):
    h ,w ,dep = frame.shape
    (x, y), (x2, y2) = box
    if y < 0:
        y2 -= y
        y = 0
    if x < 0:
        x2 -= x
        x = 0
    if x2 > w:
        x -= x2-w
        x2 = w
    if y2 > h:
        y -= y2-h
        y2 = h
    if x < 0 or y < 0 or x2 > w or y2 > h:
        return False
    else:
        box = (int(x), int(y)), (int(x2), int(y2))
        return box
def draw_bounding_box(frame, bounding_box_list):
    for box in bounding_box_list:
        (x, y), (x2, y2) = box
        cv2.rectangle(frame, (x, y), (x2, y2) , color=(255,255,255), thickness=5)
    return frame
def crop_bounding_box(frame, bounding_box_list):
    frame_list = []
    for box in bounding_box_list:
        (x, y), (x2, y2) = box
        f = frame[y:y2,x:x2,:]
        frame_list.append(f)
    return frame_list

def demo(video_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=None, skip_beg_frame=0):
    # cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture(video_path)
    for i in range(skip_beg_frame):
        ret, frame = cap.read()
    while(cap.isOpened()):

        ret, frame = cap.read()
        faces = detect_face(frame, return_max_area_face)
        if factor:
            bounding_box_list = bounding_face(frame, faces, factor, constraint)
        elif fixed_size:
            bounding_box_list = bounding_face_fixed_size(frame, faces, fixed_size, constraint)
        else:
            print 'error'
            break
        frame = draw_bounding_box(frame, bounding_box_list)
        cv2.imshow('frame',frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('k'):
            for i in range(random.randint(0,200)):
                ret, frame = cap.read()


    cap.release()
    cv2.destroyAllWindows()

def crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=None, skip_beg_frame=0):
    cap = cv2.VideoCapture(video_path)
    frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with tqdm(total=frame_length) as pbar:



        for i in range(skip_beg_frame):
            ret, frame = cap.read()
            pbar.update(1)
        existed_jpgs = [f for f in os.listdir(ouptut_path) if f.endswith('.jpg')]
        idx = len(existed_jpgs)
        print 'start from idx : %d' % idx
        while(cap.isOpened()):

            ret, frame = cap.read()
            if not ret:
                break
            pbar.update(1)
            faces = detect_face(frame, return_max_area_face)
            if factor:
                bounding_box_list = bounding_face(frame, faces, factor, constraint)
            elif fixed_size:
                bounding_box_list = bounding_face_fixed_size(frame, faces, fixed_size, constraint)
            else:
                print 'error'
                break
            frame_list = crop_bounding_box(frame, bounding_box_list)
            for frame in frame_list:
                out_path = join(ouptut_path,'%d.jpg' % idx)
                idx += 1
                write_ret = cv2.imwrite(out_path,frame)
                if not write_ret:
                    write_ret = cv2.imwrite(out_path,frame)
                    print frame
                    print out_path                    
    #             skip 10 frame
                for i in range(10):
                    ret, frame = cap.read()
                    pbar.update(1)
    print 'Cropping : Done'
    
def demo_and_crop(ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=None, skip_beg_frame=0):
    cap = cv2.VideoCapture(0)
    frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    existed_jpgs = [f for f in os.listdir(ouptut_path) if f.endswith('.jpg')]
    idx = len(existed_jpgs)
    print 'start from idx : %d' % idx
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        faces = detect_face(frame, return_max_area_face)
        if factor:
            bounding_box_list = bounding_face(frame, faces, factor, constraint)
        elif fixed_size:
            bounding_box_list = bounding_face_fixed_size(frame, faces, fixed_size, constraint)
        else:
            print 'error'
            break
        b_frame = draw_bounding_box(frame, bounding_box_list)
        cv2.imshow('frame',b_frame)

        frame_list = crop_bounding_box(frame, bounding_box_list)
        for frame in frame_list:
            out_path = join(ouptut_path,'%d.jpg' % idx)
            idx += 1
            write_ret = cv2.imwrite(out_path,frame)
            if not write_ret:
                write_ret = cv2.imwrite(out_path,frame)
                
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print 'Done'











In [3]:

# demo crop
skip_beg_frame = 0

ouptut_path = join('./','buf_data','lin')

fixed_size = 450
factor = None
demo_and_crop(ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)





start from idx : 0
Done


In [21]:
# 蔡英文520就職典禮
def constrain_1(box):
    (x, y), (x2, y2) = box
    return x > 450 and x2 < 1000
skip_beg_frame = 200
video_path = join('./','buf_data','520_TW_president_speech.mp4')

ouptut_path = join('./','buf_data','president')

fixed_size = 320
factor = None

demo(video_path, constraint=constrain_1, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
# crop_video(video_path, ouptut_path, constraint=constrain_1, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)




In [9]:
# 蔡英文CSIS演說
video_path = join('./','buf_data','TW_president_CSIS.mp4')
skip_beg_frame = 1000
ouptut_path = join('./','buf_data','president')
fixed_size = 320
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)



In [11]:
# 蔡英文 president_2016
video_path = join('./','buf_data','president_2016.mp4')
skip_beg_frame = 7000
fixed_size = 320
factor = None
ouptut_path = join('./','buf_data','president')

demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
# crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)



In [32]:
# 蔡英文 TV show
video_path = join('./','buf_data','president_TV.mp4')
skip_beg_frame = 000
fixed_size = 600
factor = None
ouptut_path = join('./','buf_data','president')

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)




  0%|          | 0/3778 [00:00<?, ?it/s]

start from idx : 7463


3784it [02:07, 29.64it/s]                          

Cropping : Done


In [38]:
# 蔡英文 speech
video_path = join('./','buf_data','president_speech.mp4')
skip_beg_frame = 200
fixed_size = 350
factor = None
ouptut_path = join('./','buf_data','president')

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)





  1%|          | 202/36320 [00:00<01:47, 336.20it/s]

start from idx : 7744


100%|█████████▉| 36317/36320 [12:01<00:00, 50.33it/s] 

Cropping : Done


In [28]:
# 館長
video_path = join('./','buf_data','fit_coach1.mp4')
skip_beg_frame = 0 
fixed_size = 500
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','fit_coach')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)


  0%|          | 0/73216 [00:00<?, ?it/s]

start from idx : 5690


73220it [50:37, 24.10it/s]                             

Cropping : Done


In [22]:
# 館長2
video_path = join('./','buf_data','fit_coach2.mp4')
skip_beg_frame = 0 
fixed_size = 460
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','fit_coach')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)



In [28]:
# 館長3
video_path = join('./','buf_data','fit_coach3.mp4')
skip_beg_frame = 500
fixed_size = 720
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','fit_coach')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)




  1%|          | 487/55070 [00:02<03:34, 254.92it/s]

start from idx : 15628


100%|██████████| 55070/55070 [27:43<00:00,  4.68it/s]  

Cropping : Done


In [3]:
# annie 1
video_path = join('./','buf_data','annie_1.mp4')
skip_beg_frame = 100
fixed_size = 450
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','annie')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)





  1%|          | 95/9970 [00:00<00:23, 422.15it/s]

start from idx : 0


9972it [04:09, 39.90it/s]                          

Cropping : Done


In [11]:
# annie 2
video_path = join('./','buf_data','annie_2.mp4')
skip_beg_frame = 100
fixed_size = 450
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','annie')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)






  2%|▏         | 102/6337 [00:00<00:25, 246.85it/s]

start from idx : 812


100%|██████████| 6337/6337 [01:31<00:00, 69.54it/s]

Cropping : Done


In [13]:
# annie 3
video_path = join('./','buf_data','annie_3.mp4')
skip_beg_frame = 100
fixed_size = 450
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','annie')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)







  1%|▏         | 91/7019 [00:00<00:16, 424.93it/s]

start from idx : 1377


100%|██████████| 7019/7019 [01:53<00:00, 62.04it/s]

Cropping : Done


In [15]:
# annie 4
video_path = join('./','buf_data','annie_4.mp4')
skip_beg_frame = 100
fixed_size = 450
factor = None

# demo(video_path, constraint=None, return_max_area_face=True, factor=factor, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)
ouptut_path = join('./','buf_data','annie')
crop_video(video_path, ouptut_path, constraint=None, return_max_area_face=True, factor=None, fixed_size=fixed_size, skip_beg_frame=skip_beg_frame)








  1%|▏         | 102/8149 [00:00<00:30, 263.20it/s]

start from idx : 1997


100%|██████████| 8149/8149 [02:01<00:00, 67.16it/s]

Cropping : Done
